works with kernel Python \[conda env:pangu] as constructed in README

run fengwu with 1 GPU and 20 GB memory (must be v100, not gp100 because gp100 only has 16 GB VRAM, but v100 has 32GB)

Make these the default casper modules `module save`

```
Currently Loaded Modules:
  1) ncarenv/24.12  (S)   3) ncarcompilers/1.0.0   5) ucx/1.17.0      7) hdf5/1.12.3    9) cudnn/9.2.0.82-12
  2) intel/2024.2.1       4) cuda/12.3.2           6) openmpi/5.0.6   8) netcdf/4.9.2  10) conda/latest
```

Inferences in conda env:ainwp is different by 0.0001 K from pangu env (after 240 hours)
conda env:ainwp is supposed to replicate realtime runs, but still 0.1 K different from realtime runs

In [2]:
import os
from pathlib import Path

import pandas as pd
import plot_ensemble
import xarray as xr
from earth2studio.data.mpas import MPAS as MPASDataSource
from earth2studio.data.mpas import MPASHybrid
from earth2studio.data.mpas_ens import MPASEnsemble
from earth2studio.io import IOBackend, NetCDF4Backend, WPSBackend
from earth2studio.models.px import GraphCastOperational, GraphCastSmall, Pangu6
from earth2studio.run import deterministic
from s3_run_pangu_ecmwf import run_inference, setup_model_sessions

SCRATCH = Path(os.getenv("SCRATCH"))
ai_models_dir = SCRATCH / "ai-models"
date = pd.to_datetime("2018042400", format="%Y%m%d%H")
# date = pd.to_datetime("2024042400", format="%Y%m%d%H")
ic = "mpas"
fhr_end = 120

In [3]:
!uname -a

Linux casper46 6.4.0-150600.23.70-default #1 SMP PREEMPT_DYNAMIC Wed Sep 10 10:54:24 UTC 2025 (225af75) x86_64 x86_64 x86_64 GNU/Linux


In [11]:
!nvidia-smi

Mon Nov 10 13:43:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40                     On  |   00000000:82:00.0 Off |                    0 |
| N/A   66C    P0            100W /  300W |   36757MiB /  46068MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
if True:
    idir = Path(
        "/glade/derecho/scratch/stoedtli/pandac/stoedtli_3dhybrid-60-60-iter_O30kmI60km_benchmark_1/CyclingFC"
    )
    mpas_datasrc = MPASHybrid(
        grid_path=idir / f"{date:%Y%m%d%H}/invariant.655362.nc",
        data_path=f"{idir}/%Y%m%d%H/mpasin.%Y-%m-%d_%H.%M.%S.nc",
        pressure_levels=[50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000],
    )

if False:
    idir = Path("/glade/campaign/mmm/parc/schwartz")
    mpas_datasrc = MPASEns(
        grid_path=idir / "MPAS/15-3km_mesh/grid_mesh/x5.6488066.grid_CONUS.nc",
        data_dir=idir / f"HWT{date.year}/mpas",
    )
if False:
    mpas_datasrc = MPASEns(
        grid_path=idir / "MPAS/15km_mesh/grid_mesh/x1.2621442.grid.nc",
        data_dir=idir / f"HWT{date.year}/mpas_15km",
    )

if False:
    idir = (
        Path(
            "/glade/derecho/scratch/stoedtli/pandac/stoedtli_3dhybrid-60-60-iter_O30kmI60km_benchmark_1/CyclingFC"
        )
        / f"{date:%Y%m%d%H}"
    )
    mpas_datasrc = MPASDataSource(
        grid_path=idir / "invariant.655362.nc",
        data_path=[
            idir / f"diag.{date:%Y-%m-%d_%H.%M.%S}.nc",
            idir / f"p_sfc.{date:%Y-%m-%d_%H.%M.%S}.nc",
        ],
    )

2025-11-10 13:27:34.889 | INFO     | earth2studio.data.mpas:_prepare_regridding_indices:104 - Loading cached regridding indices from /glade/u/home/ahijevyc/.cache/earth2studio/mpas_hybrid/invariant.655362_0.25x0.25.npz


In [6]:
model_class = GraphCastOperational
model = model_class.load_model(model_class.load_default_package())

INFO:2025-11-10 13:28:12,793:jax._src.xla_bridge:925: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2025-11-10 13:28:12,793 - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-11-10 13:28:12,958:jax._src.xla_bridge:925: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2025-11-10 13:28:12,958 - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [ ]:
mpas_datasrc(
    ["20180423T18"],
    ["t1000"],
).plot(col="time", robust=True)

In [8]:
nsteps = 1
ofile = SCRATCH / f"tmp/test_GRAP:{date:%Y-%m-%d_%H}.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = WPSBackend(SCRATCH / "tmp", map_source=model.__class__.__name__, static_fields=["lsm", "z"])
#io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
deterministic([date], nsteps, model, mpas_datasrc, io)
io.close()

2025-11-10 13:29:09.893 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-11-10 13:29:09.914 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda
2025-11-10 13:29:09.947 | INFO     | earth2studio.data.mpas:_load_and_process:289 - Requesting source variables for time 2018-04-23T18:00:00.000000000: ['rainnc', 'ter', 'theta', 'pressure_p', 'uReconstructZonal', 'pressure_base', 'w', 'zgrid', 'surface_pressure', 'u10', 't2m', 'rainc', 'v10', 'uReconstructMeridional', 'qv', 'landmask']
2025-11-10 13:29:09.958 | INFO     | earth2studio.utils.time:xtime:123 - Decoding 'initial_time' variable.
2025-11-10 13:29:11.584 | INFO     | earth2studio.utils.time:xtime:142 - Decoding 'xtime' (valid time) variable.
2025-11-10 13:29:15.353 | INFO     | earth2studio.lexicon.mpas:derive_variables:418 - Deriving pressure vertical velocity from geometric w.
2025-11-10 13:29:15.707 | INFO     | earth2studio.lexicon.mpas:derive_variables:465 - Deriving mslp from sf


Running inference:   0%|          | 0/2 [00:00<?, ?it/s]
                                                     
Running inference:  50%|█████     | 1/2 [00:05<00:05,  5.99s/it]

2025-11-10 13:30:01.076 | INFO     | earth2studio.io.wps:write:101 - Capturing static fields from initial time step: ['lsm', 'z']


2025-11-10 13:30:01,862 - WARNING - Skipping gradient checkpointing for sequence length of 1

Running inference: 100%|██████████| 2/2 [01:33<00:00, 46.60s/it]


2025-11-10 13:31:28.370 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete
2025-11-10 13:31:28.374 | WARNING  | earth2studio.io.wps:close:206 - Output file /glade/derecho/scratch/ahijevyc/tmp/GRAP:2018-04-24_00 already exists and will be overwritten.
2025-11-10 13:31:28.397 | INFO     | earth2studio.io.wps:close:223 - Overriding variables with stored static fields.
2025-11-10 13:31:28.398 | INFO     | earth2studio.io.wps:close:237 - Writing final forecast step for time 2018-04-24_00:00:00 (F006) to /glade/derecho/scratch/ahijevyc/tmp/GRAP:2018-04-24_00
2025-11-10 13:31:28.748 | INFO     | earth2studio.io.wps:close:299 - WPSBackend closed. Final output: /glade/derecho/scratch/ahijevyc/tmp/GRAP:2018-04-24_00


In [ ]:
from metpy.constants import Re
Re

In [ ]:
ort_session_24, ort_session_6 = setup_model_sessions(ai_models_dir)

In [ ]:
da

In [ ]:
output_dir = Path(f"/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather/{date:%Y%m%d%H}")

da = ds.rename(variable="channel")
inferences = run_inference(da, ort_session_24, ort_session_6, fhr_end)
for fcst in inferences:
    fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
    output_filename = os.path.join(output_dir, f"pangu_{ic}_hybrid_pred_{fhr:03.0f}.nc")
    print(output_filename)
    # zlib compression can reduce by 1/3 but takes long time (versus almost instantaneous)
    fcst.to_netcdf(output_filename)

In [ ]:
output_dir = Path(f"/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather/{date:%Y%m%d%H}")

for member in ds.member.data:
    da = ds.sel(member=member).rename(variable="channel").rename(Time="time")
    all_exist = True
    for fhr in range(6, fhr_end+1, 6):
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        if not os.path.exists(output_filename):
            print(output_filename, 'no exists')
            all_exist = False
            break
    if all_exist:
        print(f"all {date} {ic}{member} exist")
        continue
    inferences = run_inference(da, ort_session_24, ort_session_6, fhr_end)
    for fcst in inferences:
        fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        print(output_filename)
        # zlib compression can reduce by 1/3 but takes long time (versus almost instantaneous)
        fcst.to_netcdf(output_filename)

In [ ]:
ifiles = []
for member in ds.member.data:
    ifiles.append(sorted(list(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))))
da = (
    xr.open_mfdataset(
        ifiles,
        combine="nested",
        concat_dim=["member", "prediction_timedelta"],
    )
    .rename(lat="latitude", lon="longitude")
    .assign_coords(member=ds.member)
    .sel(channel="z500")
    .rename(__xarray_dataarray_variable__="z", prediction_timedelta="step")
    .squeeze(dim="init_time")
)
da

In [ ]:
import argparse
args = argparse.Namespace
args.ic = ic.upper()
args.model = "panguweather"
fig = plot_ensemble.plot_forecast_grid(args, da.sortby("member"), plotdays=range(1, fhr_end//24+1))

In [ ]:
member = 4
inferences = xr.open_mfdataset(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))
inferences = inferences.sel(lat=slice(60,20), lon=slice(220, 300))
inferences.sel(channel="z500").squeeze().__xarray_dataarray_variable__.plot(col="prediction_timedelta", col_wrap=6)

In [ ]:
inferences